# <font color="blue"> MBA em Ciência de Dados</font>
# <font color="blue">Técnicas Avançadas para Captura e Tratamento de Dados</font>

## <font color="blue"> Web Scraping </font>

## <font color="blue">Exercícios </font> 

**Material Produzido por Damares Resende**<br>
**Cemeai - ICMC/USP São Carlos**

---

In [1]:
import requests as rq

### Exercício 1)
Visite o site [https://www.cvc.com.br/destinos/](https://www.cvc.com.br/destinos/). Verifique se a requisição foi bem sucedida e o o tipo de dado retornado.

In [2]:
page = rq.get('https://www.cvc.com.br/destinos/')

print('Status do request %s' % page.status_code)
print('Tipo do conteúdo: %s' % page.headers['Content-Type'])

Status do request 200
Tipo do conteúdo: text/html; charset=utf-8


### Exercício 2)
Considere a requisição feita na página anterior, extraia a URL de todos os links dos pontos turísticos brasileiros, e salve-as em um dicionário aonde a chave é o destino, e o valor é a URL.

In [3]:
from bs4 import BeautifulSoup

root_url = 'https://www.cvc.com.br'
page = rq.get(root_url + '/destinos/')

destinations = dict()
if page.status_code == 200:
    soup = BeautifulSoup(page.text, 'html.parser')

    for item in soup('li', {'class': 'jss90 jss137 jss149 jss82'}):
        item_details = item.find('figure').find('a')
        destination = item_details.find('h2').text
        country = item_details.find('h3').text
        url = item_details.get('href')
        
        print(f'Destino: {destination}, Pais: {country}, URL: {url}')
        
        if country.strip() == 'Brasil':
            destinations[destination] = root_url + url
        
print(destinations)

Destino: Aruba, Pais: Aruba, URL: /destinos/aruba-paises-baixos
Destino: Natal, Pais: Brasil, URL: /destinos/natal-brasil
Destino: Maceió, Pais: Brasil, URL: /destinos/maceio-brasil
Destino: Fortaleza, Pais: Brasil, URL: /destinos/fortaleza-brasil
Destino: Foz do Iguaçu, Pais: Brasil, URL: /destinos/foz-do-iguacu-brasil
Destino: Serra Gaúcha, Pais: Brasil, URL: /destinos/serra-gaucha-brasil
Destino: Paris, Pais: França, URL: /destinos/paris-franca
Destino: Londres, Pais: Inglaterra, URL: /destinos/londres-inglaterra
Destino: Trancoso, Pais: Brasil, URL: /destinos/trancoso-brasil
Destino: Santa Cruz Cabrália, Pais: Brasil, URL: /destinos/santa-cruz-cabralia-brasil
Destino: Lisboa, Pais: Portugal, URL: /destinos/lisboa-portugal
Destino: Bolívia, Pais: América do Sul, URL: /destinos/bolivia
Destino: Uruguai, Pais: Uruguai, URL: /destinos/uruguai
Destino: São Paulo, Pais: Brasil, URL: /destinos/sao-paulo-brasil
Destino: Bora Bora, Pais: Polinésia Francesa, URL: /destinos/bora-bora-polinesi

### Exercício 3)
Crie um dicionário para cada destino encontrado no exercício 2 com as informações básicas sobre ele, como os campos: "Sobre o Destino", "Gastronomia", "Diversão" e etc. A chave é o nome do destino, o valor é um novo dicionário com as informações encontradas aonde a chave é o nome do campo e o valor é o texto.

**DICA**: Note que esses campos variam em quantidade e nome dependendo do destino.

In [4]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

option = Options()
option.headless = True
driver = webdriver.Chrome(options = option)

print('starting')

details = dict()
for destino, url in destinations.items():
    dest_page = rq.get(url)
    driver.get(url)
    
    if dest_page.status_code != 200:
        continue
        
    soup = BeautifulSoup(dest_page.text, 'html.parser')
    
    tabs = soup('nav', {'class': 'jss130'})
    if not tabs:
        tabs = soup('nav', {'class': 'jss129'})
        if not tabs:
            tabs = soup('nav', {'class': 'jss128'})

    for tab in tabs:
        info = {title.text.strip(): '' for title in tab.find_all('span') if title.text.strip()}
        details[destino] = info
    
    tabs = list(details[destino].keys())
    for i in range(len(tabs)):
        login_btn = driver.find_element_by_xpath('//*[@id="root"]/main[1]/section[2]/article[1]/div[1]/nav[1]')
        time.sleep(.2)
        login_btn.find_element_by_xpath('//*/button[%d]' % (i + 1)).click()
        
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for content in soup('div', {'class': 'jss114'}):
        for i, text in enumerate(content.find_all('div', class_='jss168')):
            details[destino][tabs[i]] = '\n'.join([p.text for p in text.find_all('p')]) 

#     break
    
driver.quit()
print(details)

print('ending')

starting
{'Natal': {'Sobre O Destino': 'Com sol praticamente o ano inteiro, a capital do Rio Grande do Norte é um dos destinos preferidos do Nordeste. Conhecida por suas praias, dunas, lagoas e a simpatia de seu povo, a viagem para Natal oferece passeios inesquecíveis como Genipabu, Pirangi, Pipa e São Miguel do Gostoso. E quando bate a fome, a gastronomia regional entra em cena com pratos a base de frutos do mar, peixes, tapioca e outras comidinhas típicas.\nGostou desse destino? Confira hotéis, pacotes e passagens aéreas para Natal.', 'Praia': 'Algumas das praias mais conhecidas da capital potiguar são Ponta Negra, onde está o cartão-postal da cidade, o Morro do Careca, duna com 120 metros de altura. A praia possui muitos hotéis, pousadas, restaurantes, bares, lojas e quiosques.\nA Praia dos Artistas é outro ponto famoso da capital e uma das mais agitadas. Com ondas altas ideais para o surfe, conta com um centro de artesanato e um mirante. Na Praia da Costeira ficam alguns dos melhor

### Exercício 4)
Análogo ao exercício anterior, crie também um dicionário com a lista de "O Que Fazer" para os três primeiros destinos encotrados.

In [6]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

option = Options()
option.headless = False
driver = webdriver.Chrome(options = option)

activities = dict()
for destino, url in list(destinations.items())[:3]:
    dest_page = rq.get(url)
    driver.get(url)

    if dest_page.status_code != 200:
        continue

    while True:
        try:
            time.sleep(.2)
            driver.find_element_by_id('tourist-attraction-section').find_element_by_id('right').click()
        except NoSuchElementException:
            break

    info = dict()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for elem in soup.find('section', id='tourist-attraction-section').find_all('div', class_='jss174'):
        title = elem.find('h3').text.strip()
        text = '\n'.join([t.text.strip() for t in elem.find_all('p')])
        info[title] = text

    if info:
        activities[destino] = info

    time.sleep(1)
    
driver.quit()
print(activities)
        
    

{'Natal': {'Passeio de bugue a Genipabu': 'O tradicional passeio em Genipabu, a 24 km de Natal, pode ser feito com ou sem emoção! O bugue também faz uma parada na Lagoa de Jacumã, para práticas esportivas como "esquibunda" e "aerobunda". Você também pode fazer um passeio de dromedário pelas dunas.\nAs praias de Cacimbinhas e do Madeiro são acessadas por uma escadaria escavada na falésia. Lá é possível avistar golfinhos no mar. Depois visita a Praia da Pipa, onde poderá fazer um passeio na Lagoa de Guaraíras e seguir de trenzinho até o Chapadão, falésia com vista para a Praia do Amor.\nDescubra falésias coloridas, praias desertas, lagoas de água doce e trechos de Mata Atlântica. Destaque para os mirantes dos Pescadores e da Praia de Baía Formosa, ideais para fotos. Faça também um passeio de bugue até a Praia de Sagi.\nFamosa pela reserva natural de mangue, as piscinas naturais que se formam na maré baixa e a criação de camarão em viveiros. Aproveite o dia para visitar a paradisíaca Prai